## Baseline model: average "dummy" regressor

L'idée est la suivante : on prend le jeu de données de Train, on estime la position cartésienne moyenne des individus qui le compose. Sur l'inférence sur le jeu de test, on donne toujours la position moyenne calculée précédemment.

In [1]:
import sys
import os
import random

sys.path.append(os.getcwd() + '/..')
sys.path.append(os.getcwd() + '/../..')

In [2]:
from get_data import load_data, load_label, fetch_particular_series, plot_series, load_particular_label

In [4]:
import matplotlib.pyplot as plt
import pyproj
from pyproj import Proj
import json

from skimage import data, transform
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, plot_matches, SIFT

import cv2
import numpy as np

In [3]:
# on commence par couper le jeu de données en train, test (pas de validation car pas de sélection d'hyperparamètres)
all_filenames = os.listdir("../../data/images")
all_ids = list(set([item.split("_")[0] for item in all_filenames]))
N = len(all_ids)
# shuffle IDs
random.shuffle(all_ids)
# 20% des données pour le test, 80% pour constituer la base de données
train_test_ratio = 0.8
X_train_ids = all_ids[:int(train_test_ratio*N)]
X_test_ids = all_ids[int(train_test_ratio*N):]
print(len(X_train_ids))
print(len(X_test_ids)) 

355
89


In [5]:
latitudes, longitudes = [], []
for image_id in X_train_ids:
    (lat, long, _) = load_particular_label(image_id, path = "../../data/labels.mat")
    inProj = Proj(init='epsg:4326') # (lat, long)
    outProj = Proj(init='epsg:2154') # Lambert 93
    x_l93, y_l93 = pyproj.transform(inProj, outProj, lat, long)
    latitudes.append(x_l93)
    longitudes.append(y_l93)

average_lat_train = np.mean(latitudes)
average_long_train = np.mean(longitudes)

C:\Users\cleme\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\cleme\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\cleme\AppData\Local\Temp\ipykernel_12356\2643093886.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-

In [6]:
print("Average x-coordinate:", average_lat_train)
print("Average y-coordinate:", average_long_train)

Average x-coordinate: 23049914.13492254
Average y-coordinate: -30503576.593386058


In [7]:
# run inference and gather losses
losses = []
for image_id in X_test_ids:
    (lat, long, _) = load_particular_label(image_id, path = "../../data/labels.mat")
    inProj = Proj(init='epsg:4326') # (lat, long)
    outProj = Proj(init='epsg:2154') # Lambert 93
    x_test_l93, y_test_l93 = pyproj.transform(inProj, outProj, lat, long)
    dist = np.sqrt(
        (x_test_l93 - average_lat_train)**2 + (y_test_l93 - average_long_train)**2
    )
    losses.append(dist)

average_loss = np.mean(losses) # in meters

C:\Users\cleme\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\cleme\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\cleme\AppData\Local\Temp\ipykernel_12356\1407275913.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-

In [8]:
print("Average distance on the test set is:", average_loss)

Average distance on the test set is: 8334.526266687752


Conclusion : le dummy regressor donne une erreur moyenne de 8km sur le jeu de données. À comparer à la lumière des performances des autres modèles.